This notebook is same as the previous one i.e., [RAG_GPT3_5_with_ChromaDB_multi_doc](https://github.com/yashasvi-shukl/GenAI/blob/2cdbdab9f6a8f5ec9ac51a66c44c8288c9d76774/RAG/1.%20RAG_GPT3_5_with_ChromaDB_multi_doc.ipynb)

The only difference is, in this notebooks I'm using Instructor and sentence-transformmers embeddings an not OpenAIEmbeddings

In [1]:
!pip -q install openai langchain tiktoken chromadb sentence_transformers InstructorEmbedding pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.6 MB/s eta 

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.340
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!wget -q https://www.dropbox.com/s/zoj9rnm7oyeaivb/new_papers.zip
!unzip -q new_papers.zip -d new_papers

## LangChain multi-doc retriever with ChromaDB

Key Points

*   Multiple Files - PDFs
*   ChromaDB - with more meta data?
*   Source info
*   gpt-3.5-turbo API
*   HuggingFace Embeddings
*   Instuctor Embeddings







In [4]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

··········


In [5]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [6]:
# Loading multiple pdf files
# The pdf files are Archive research papers

# loader = TextLoader('single_text_file.txt)  # To load single text file

loader = DirectoryLoader(path = "/content/new_papers/new_papers",
                         glob = "./*.pdf",
                         loader_cls = PyPDFLoader)

documents = loader.load()

In [7]:
len(documents)

142

In [8]:
# Splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
texts = text_splitter.split_documents(documents)

In [9]:
len(texts)

659

In [10]:
texts[0]

Document(page_content='FlashAttention : Fast and Memory-Eﬃcient Exact Attention\nwith IO-Awareness\nTri Daoy, Daniel Y. Fuy, Stefano Ermony, Atri Rudraz, and Christopher Réy\nyDepartment of Computer Science, Stanford University\nzDepartment of Computer Science and Engineering, University at Buﬀalo, SUNY\n{trid,danfu}@cs.stanford.edu ,ermon@stanford.edu ,atri@buffalo.edu ,\nchrismre@cs.stanford.edu\nJune 24, 2022\nAbstract\nTransformers are slow and memory-hungry on long sequences, since the time and memory complexity\nof self-attention are quadratic in sequence length. Approximate attention methods have attempted\nto address this problem by trading oﬀ model quality to reduce the compute complexity, but often do\nnot achieve wall-clock speedup. We argue that a missing principle is making attention algorithms IO-\naware—accounting for reads and writes between levels of GPU memory. We propose FlashAttention ,\nan IO-aware exact attention algorithm that uses tiling to reduce the number of 

### Using HuggingFace Embeddings

In [11]:
# Sentence Transformers Embeddings

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceEmbeddings(model_name = model_name)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [12]:
# Instructor Embeddings

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name = "hkunlp/instructor-xl",
    model_kwargs = {"device": "cuda"}
)

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


### Creating DB

In [13]:
# Embed and store the text in db
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

# Here is the new embedding i.e., instruct embedding. You can use any sentence transfromer embeddigs as well
vectordb = Chroma.from_documents(
    documents = documents,
    embedding = instructor_embeddings,
    persist_directory = persist_directory
)

In [14]:
# persist the db on the disk
vectordb.persist()
vectordb = None

In [15]:
# Noew we can load the persisted database from the disk
vectordb = Chroma(
    persist_directory = persist_directory,
    embedding_function = instructor_embeddings
)

### Setting up Retriever

In [16]:
retriever = vectordb.as_retriever()

In [17]:
docs = retriever.get_relevant_documents("How does Flash Attention works?")

In [18]:
len(docs)

4

In [19]:
docs

[Document(page_content='FlashAttention : Fast and Memory-Eﬃcient Exact Attention\nwith IO-Awareness\nTri Daoy, Daniel Y. Fuy, Stefano Ermony, Atri Rudraz, and Christopher Réy\nyDepartment of Computer Science, Stanford University\nzDepartment of Computer Science and Engineering, University at Buﬀalo, SUNY\n{trid,danfu}@cs.stanford.edu ,ermon@stanford.edu ,atri@buffalo.edu ,\nchrismre@cs.stanford.edu\nJune 24, 2022\nAbstract\nTransformers are slow and memory-hungry on long sequences, since the time and memory complexity\nof self-attention are quadratic in sequence length. Approximate attention methods have attempted\nto address this problem by trading oﬀ model quality to reduce the compute complexity, but often do\nnot achieve wall-clock speedup. We argue that a missing principle is making attention algorithms IO-\naware—accounting for reads and writes between levels of GPU memory. We propose FlashAttention ,\nan IO-aware exact attention algorithm that uses tiling to reduce the number of

In [26]:
retriever = vectordb.as_retriever(search_kwargs = {"k":3})

### Building up the Chain

In [27]:
chain = RetrievalQA.from_chain_type(
    llm = OpenAI(),
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True
)

In [33]:
import textwrap

def wrap_text_preserve_newlines(text, width=100):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [29]:
query = "How does Flash Attention works?"
llm_response = chain(query)
process_llm_response(llm_response)

 Flash Attention works by using tiling to reduce the number of memory reads/writes between GPU high bandwidth
memory (HBM) and GPU on-chip SRAM. It also accounts for reads and writes to different levels of fast and slow
memory and focuses on IO complexity rather than FLOP reduction.


Sources:
/content/new_papers/new_papers/Flash-attention.pdf
/content/new_papers/new_papers/Flash-attention.pdf
/content/new_papers/new_papers/Flash-attention.pdf


In [34]:
llm_response = chain("What is tiling in flash attention?")
process_llm_response(llm_response)

 Tiling in FlashAttention is a technique used to reduce the number of memory reads/writes between
GPU high bandwidth memory (HBM) and GPU on-chip SRAM. It involves breaking down the attention matrix
into smaller blocks which are processed separately in order to reduce the amount of data that needs
to be accessed at one time.


Sources:
/content/new_papers/new_papers/Flash-attention.pdf
/content/new_papers/new_papers/Flash-attention.pdf
/content/new_papers/new_papers/Flash-attention.pdf


In [35]:
query = "What is toolformer?"
llm_response = chain(query)
process_llm_response(llm_response)

 Toolformer is a model trained to decide which APIs to call, when to call them, what arguments to
pass, and how to best incorporate the results into future token prediction. This is done in a self-
supervised way, requiring nothing more than a handful of demonstrations for each API.


Sources:
/content/new_papers/new_papers/toolformer.pdf
/content/new_papers/new_papers/toolformer.pdf
/content/new_papers/new_papers/toolformer.pdf


## Since here follow the notebook linked above